# Pharmaceutical Sales prediction across multiple stores
In this project, we use the data obtained from Rossman Pharmacueticals to conduct  sales prediction across their store 6 weeks ahead. This notebook conducts data preprocessing  in order to understand the data better. This project involves a time series data analysis

## Importing the required packages and modules

In [41]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../scripts')
from load_data import LoadData
import matplotlib.pyplot as plt
import seaborn as sns
import logging
from sklearn.impute import SimpleImputer
from sklearn import preprocessing 
from datetime import datetime

## Loading the data into dataframes

In [30]:
loader = LoadData()
train_df = loader.read_csv('../data/train.csv')
test_df= loader.read_csv('../data/test.csv')
store_df= loader.read_csv('../data/store.csv')

C:\Users\User\anaconda3\envs\envMysql\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [31]:
train_df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [32]:
test_df.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


In [33]:
store_df.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


## Merging the data 
The data was provide in 4 separate csv files:
<ol>
    <li>train.csv - This contains data exclusive to the sales made by a particular store. We will use this data to train the models we build </li>
    <li>test.csv - This contains data exclusive to the sales made by a particular store. We will use this data to test the models we build </li>
    <li>train.csv - This contains data that describes the stores </li>
    <li>sample_submission.csv - Contains sample submissions that gives the format of submission of the results we get </li>
</ol>

We need to merge the train dataset with the stores dataset and also merge the test dataset with the stores dataset

In [34]:
combined_train_df = pd.merge(train_df, store_df, on='Store')
combined_train_df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [35]:
combined_test_df = pd.merge(test_df, store_df, on='Store')
combined_test_df.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,1,4,2015-09-17,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,857,1,3,2015-09-16,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,1713,1,2,2015-09-15,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,2569,1,1,2015-09-14,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,3425,1,7,2015-09-13,0.0,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


## Data Preprocessing

### Convert the dates provided in the datasets into datetime formats

In [36]:

#converting date from string into datetime for train and test
combined_test_df['Date'] = combined_test_df['Date'].map(lambda x: datetime.strptime(x, '%Y-%m-%d'))
combined_train_df['Date'] = combined_train_df['Date'].map(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [37]:
#Extracting weekdays
combined_test_df['isWeekend'] = combined_test_df['DayOfWeek'].map(lambda x: 0 if x < 6 else 1)
combined_train_df['isWeekend'] = combined_train_df['DayOfWeek'].map(lambda x: 0 if x < 6 else 1)


In [38]:
df = combined_train_df.query('StateHoliday == "a"')
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,isWeekend
57,1,4,2015-06-04,0,0,0,1,a,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,0
67,1,1,2015-05-25,0,0,0,0,a,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,0
78,1,4,2015-05-14,0,0,0,0,a,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,0
91,1,5,2015-05-01,0,0,0,1,a,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,0
211,1,4,2015-01-01,0,0,0,0,a,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,0


### Adding Year, Day and Month as three separate features to train and test set

In [14]:

combined_train_df['Year'] = combined_train_df['Date'].map(lambda x: x.year)
combined_train_df['Month'] = combined_train_df['Date'].map(lambda x: x.month)
combined_train_df['Day'] = combined_train_df['Date'].map(lambda x: x.day)

combined_test_df['Year'] = combined_test_df['Date'].map(lambda x: x.year)
combined_test_df['Month'] = combined_test_df['Date'].map(lambda x: x.month)
combined_test_df['Day'] =combined_test_df['Date'].map(lambda x: x.day)

### Transform StateHoliday column 
StateHoliday column has values 0 & "0", So, we need to change values with 0 to "0"

In [43]:
combined_test_df["StateHoliday"].loc[combined_test_df["StateHoliday"] == 0] = "0"
combined_train_df["StateHoliday"].loc[combined_train_df["StateHoliday"] == 0] = "0"

C:\Users\User\anaconda3\envs\envMysql\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


### Set holidays
check if date is holiday

In [16]:
combined_test_df['isHoliday'] = combined_test_df['StateHoliday'].map(lambda x: 0 if x == "0" else 1)
combined_train_df['isHoliday'] = combined_train_df['StateHoliday'].map(lambda x: 0 if x == "0" else 1)

In [17]:
combined_train_df['isHoliday'].value_counts()

0    986159
1     31050
Name: isHoliday, dtype: int64

In [22]:
def setMonthPeriod(date):
    if date.day < 11: 
        return 'start month'
    elif date.day <= 21:
        return 'mid month'
    else:
        return 'end month'

### Set start, mid and end months

In [27]:
def setMnth(df):
    df['month_period'] = df['Date'].apply(setMonthPeriod)
    return df

In [28]:
setMnth(combined_test_df).head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year,Month,Day,isHoliday,month_period
0,1,1,4,2015-09-17,1.0,1,0,0,c,a,...,2008.0,0,NaN,NaN,NaN,2015,9,17,0,mid month
1,857,1,3,2015-09-16,1.0,1,0,0,c,a,...,2008.0,0,NaN,NaN,NaN,2015,9,16,0,mid month
2,1713,1,2,2015-09-15,1.0,1,0,0,c,a,...,2008.0,0,NaN,NaN,NaN,2015,9,15,0,mid month
3,2569,1,1,2015-09-14,1.0,1,0,0,c,a,...,2008.0,0,NaN,NaN,NaN,2015,9,14,0,mid month
4,3425,1,7,2015-09-13,0.0,0,0,0,c,a,...,2008.0,0,NaN,NaN,NaN,2015,9,13,0,mid month


In [23]:
combined_train_df['month_period'] = combined_train_df['Date'].apply(setMonthPeriod)
combined_train_df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year,Month,Day,isHoliday,month_period
0,1,5,2015-07-31,5263,555,1,1,0,1,c,...,2008.0,0,NaN,NaN,NaN,2015,7,31,0,end month
1,1,4,2015-07-30,5020,546,1,1,0,1,c,...,2008.0,0,NaN,NaN,NaN,2015,7,30,0,end month
2,1,3,2015-07-29,4782,523,1,1,0,1,c,...,2008.0,0,NaN,NaN,NaN,2015,7,29,0,end month
3,1,2,2015-07-28,5011,560,1,1,0,1,c,...,2008.0,0,NaN,NaN,NaN,2015,7,28,0,end month
4,1,1,2015-07-27,6102,612,1,1,0,1,c,...,2008.0,0,NaN,NaN,NaN,2015,7,27,0,end month


In [25]:
combined_train_df['month_period'].value_counts()

mid month      368335
start month    334849
end month      314025
Name: month_period, dtype: int64

## Encoding categorical columns 

In [44]:
numerical_column = combined_train_df.select_dtypes(exclude="object").columns.tolist()
categorical_column = combined_train_df.select_dtypes(include="object").columns.tolist()
print("Numerical Columns:", numerical_column)
print("****************")
print("Categorical Columns:", categorical_column)

Numerical Columns: ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'SchoolHoliday', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'isWeekend']
****************
Categorical Columns: ['StateHoliday', 'StoreType', 'Assortment', 'PromoInterval']


In [45]:
label_encoder = preprocessing.LabelEncoder()
for column in categorical_column:
    combined_train_df[column]= label_encoder.fit_transform(combined_train_df[column])
combined_train_df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,isWeekend
0,1,5,2015-07-31,5263,555,1,1,0,1,2,0,1270.0,9.0,2008.0,0,NaN,NaN,3,0
1,1,4,2015-07-30,5020,546,1,1,0,1,2,0,1270.0,9.0,2008.0,0,NaN,NaN,3,0
2,1,3,2015-07-29,4782,523,1,1,0,1,2,0,1270.0,9.0,2008.0,0,NaN,NaN,3,0
3,1,2,2015-07-28,5011,560,1,1,0,1,2,0,1270.0,9.0,2008.0,0,NaN,NaN,3,0
4,1,1,2015-07-27,6102,612,1,1,0,1,2,0,1270.0,9.0,2008.0,0,NaN,NaN,3,0
